In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 1. 브랜드명 리스트

In [ ]:
url = 'https://kream.co.kr/search?category_id=34&sort=popular&per_page=40'

In [ ]:
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)

# 브랜드 필터 선택
icons = driver.find_elements(By.CLASS_NAME,'ico_box') 
icons[1].click()
time.sleep(3)

# 브랜드 이름 불러오기
brands = []
for i in range(1,175,1):
    xpath = f'//*[@id="__layout"]/div/div[2]/div[5]/div[1]/div[3]/div[2]/ul/li[{i}]/a'
    brand = driver.find_element(By.XPATH,xpath)
    brand_name = brand.text
    brands.append(brand_name)

In [ ]:
len(brands)

174

In [ ]:
kream_brand = pd.DataFrame(columns=['brand'])
kream_brand['brand'] = brands
kream_brand.head()

,brand
0,A.P.C.
1,A.P.C. Golf
2,Acne Studios
3,Ader Error
4,Adidas


In [ ]:
kream_brand.to_csv('Data/크림_브랜드명_리스트.csv',index=False,encoding='utf-8-sig')

In [ ]:
kream_brand = pd.read_csv('Data/크림_브랜드명_리스트.csv')
brands = kream_brand['brand'].tolist()
len(brands)

174

# 2. 브랜드 별 검색

In [ ]:
product_df = pd.DataFrame(columns=['검색어','브랜드','제품명','제품한글명','가격','거래량','관심수','url'])
product_df

,검색어,브랜드,제품명,제품한글명,가격,거래량,관심수,url


## 2-1. 첫 번째 브랜드 테스트

In [ ]:
url = f'https://kream.co.kr/search?keyword={brands[0]}&category_id=34'

In [ ]:
driver = webdriver.Chrome()
driver.get(url)
time.sleep(3)

In [ ]:
# 로그인
driver.find_element(By.XPATH,'//*[@id="__layout"]/div/div[1]/div[1]/div/ul/li[4]/a').click()
time.sleep(2)
input_list = driver.find_elements(By.CLASS_NAME,'input_txt')
input_list[0].send_keys('id')
input_list[1].send_keys('pwd')
time.sleep(1)
driver.find_element(By.CLASS_NAME,'login_btn_box').click()
time.sleep(2)

In [ ]:
products = driver.find_elements(By.CLASS_NAME,'search_result_item')
len(products)

2

In [ ]:
print(products[0].find_element(By.CLASS_NAME,'brand').text)
print(products[0].find_element(By.CLASS_NAME,'name').text)
print(products[0].find_element(By.CLASS_NAME,'translated_name').text)
print(products[0].find_element(By.CLASS_NAME,'amount').text)
print(products[0].find_element(By.CLASS_NAME,'status_value').text)
print(products[0].find_element(By.CLASS_NAME,'wish_figure').text)
print(products[0].find_element(By.TAG_NAME,'a').get_attribute('href'))

Nike
Nike x A.P.C. Dunk High '08 SP Beige
나이키 x 아페쎄 덩크 하이 '08 SP 베이지
280,000원
거래 1
57
https://kream.co.kr/products/34916


## 2-2. 스크롤 테스트

In [ ]:
url = 'https://kream.co.kr/search?category_id=34&keyword=Acne Studios&sort=popular&per_page=40'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(3)

# 스크롤 끝까지 내리기 
scroll_location = driver.execute_script("return document.body.scrollHeight")

while True:
	#현재 스크롤의 가장 아래로 내림
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
       
    #전체 스크롤이 늘어날 때까지 대기
    time.sleep(2)
    
    #늘어난 스크롤 높이
    scroll_height = driver.execute_script("return document.body.scrollHeight")

    #늘어난 스크롤 위치와 이동 전 위치 같으면(더 이상 스크롤이 늘어나지 않으면) 종료
    if scroll_location == scroll_height:
    	break
	    
    #같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
    else:
    	#스크롤 위치값을 수정
    	scroll_location = driver.execute_script("return document.body.scrollHeight")

In [ ]:
products = driver.find_elements(By.CLASS_NAME,'search_result_item') # 검색 결과 
len(products)

74

## 2-2. 제품 가져오기

In [ ]:
# 로그인한 상태로 반복문 진입
url = 'https://kream.co.kr/search?category_id=34&keyword=A.P.C.&sort=popular&per_page=40'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(3)
# 로그인
id = 'id'
password = 'pwd'
driver.find_element(By.XPATH,'//*[@id="__layout"]/div/div[1]/div[1]/div/ul/li[4]/a').click() # 로그인 버튼 클릭
time.sleep(2)
input_list = driver.find_elements(By.CLASS_NAME,'input_txt') #아이디/비밀번호 입력창
input_list[0].send_keys(id) #아이디 입력
input_list[1].send_keys(password) #비밀번호 입력
time.sleep(1)
driver.find_element(By.CLASS_NAME,'login_btn_box').click() # 로그인 버튼 클릭
time.sleep(2)

In [ ]:
idx = 0 # df 저장 인덱스 

# driver = webdriver.Chrome()

for brand in tqdm(brands):
    url = f'https://kream.co.kr/search?category_id=34&keyword={brand}&sort=popular&per_page=40'
    driver.get(url)
    time.sleep(3)

    # 스크롤 끝까지 내리기
    scroll_location = driver.execute_script("return document.body.scrollHeight")

    while True:
        #현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        #전체 스크롤이 늘어날 때까지 대기
        time.sleep(2)
        #늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        #늘어난 스크롤 위치와 이동 전 위치 같으면(더 이상 스크롤이 늘어나지 않으면) 종료
        if scroll_location == scroll_height:
            break
        #같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            #스크롤 위치값을 수정
            scroll_location = driver.execute_script("return document.body.scrollHeight")

    products = driver.find_elements(By.CLASS_NAME,'search_result_item') # 검색 결과 

    # 검색 결과 제품 저장 
    for product in tqdm(products):
        product_df.loc[idx,'검색어'] = brand # 검색어
        product_df.loc[idx,'브랜드'] = product.find_element(By.CLASS_NAME,'brand').text # 브랜드명
        product_df.loc[idx,'제품명'] = product.find_element(By.CLASS_NAME,'name').text # 제품명
        trans_name_info = product.find_elements(By.CLASS_NAME,'translated_name')
        if len(trans_name_info)==0: # 한글명이 없는 경우 Null 처리 
            product_df.loc[idx,'제품한글명'] = np.NaN
        else:
            product_df.loc[idx,'제품한글명'] = product.find_element(By.CLASS_NAME,'translated_name').text #제품번역명
        product_df.loc[idx,'가격'] = product.find_element(By.CLASS_NAME,'amount').text # 가격
        trading_info = product.find_elements(By.CLASS_NAME,'status_value') # 거래량 
        if len(trading_info)==0: # 거래량이 없는 경우 Null 처리 
            product_df.loc[idx,'거래량'] = np.NaN
        else:
            product_df.loc[idx,'거래량'] = product.find_element(By.CLASS_NAME,'status_value').text
        product_df.loc[idx,'관심수'] = product.find_element(By.CLASS_NAME,'wish_figure').text # 관심 수 
        product_df.loc[idx,'url'] = product.find_element(By.TAG_NAME,'a').get_attribute('href') # 제품 페이지 url

        idx = idx + 1 # 저장 인덱스 증가 
    
    

100%|██████████| 2/2 [00:00<00:00,  5.41it/s]
0it [00:00, ?it/s]/174 [00:08<24:52,  8.63s/it]
0it [00:00, ?it/s]/174 [00:15<21:01,  7.33s/it]
100%|██████████| 33/33 [00:05<00:00,  5.59it/s]
0it [00:00, ?it/s]/174 [00:33<25:55,  9.15s/it]
100%|██████████| 13/13 [00:02<00:00,  4.80it/s]
0it [00:00, ?it/s]/174 [01:16<29:34, 10.69s/it]
0it [00:00, ?it/s]/174 [01:23<26:07,  9.50s/it]
0it [00:00, ?it/s]0/174 [01:30<23:25,  8.57s/it]
100%|██████████| 5/5 [00:00<00:00,  6.05it/s]it]
0it [00:00, ?it/s]2/174 [01:43<20:44,  7.68s/it]
0it [00:00, ?it/s]3/174 [01:50<19:38,  7.32s/it]
0it [00:00, ?it/s]4/174 [01:56<18:37,  6.99s/it]
100%|██████████| 291/291 [00:57<00:00,  5.03it/s]
0it [00:00, ?it/s]9/174 [04:17<1:27:57, 34.05s/it]
0it [00:00, ?it/s]0/174 [04:24<1:06:24, 25.88s/it]
100%|██████████| 9/9 [00:01<00:00,  5.98it/s]it]
0it [00:00, ?it/s]3/174 [05:00<42:20, 16.83s/it]
100%|██████████| 28/28 [00:04<00:00,  5.67it/s]]
0it [00:00, ?it/s]7/174 [05:58<37:36, 15.35s/it]
100%|██████████| 40/40 [0

In [ ]:
product_df['검색어'].value_counts()

Nike           8363
Jordan         3032
New Balance    1657
Vans           1007
Converse        837
               ... 
Hidden NY         1
Telfar            1
Starbucks         1
Our Legacy        1
Vinyl             1
Name: 검색어, Length: 96, dtype: int64

In [ ]:
product_df['브랜드'].value_counts()

Nike                   8926
Jordan                 3107
New Balance            1787
Vans                   1117
Converse                988
                       ... 
Jil Sander                2
Salvatore Ferragamo       2
Chanel                    1
Valentino                 1
Kenzo                     1
Name: 브랜드, Length: 65, dtype: int64

In [ ]:
product_df.tail()

,검색어,브랜드,제품명,제품한글명,가격,거래량,관심수,url
20179,Yeezy,Adidas,(Kids) Adidas Yeezy Boost 350 V2 Lundmark,NaN,-,NaN,8,https://kream.co.kr/products/21255
20180,Yeezy,Adidas,(W) Adidas Yeezy 950 Moonrock,NaN,-,NaN,3,https://kream.co.kr/products/5831
20181,Yeezy,Adidas,(W) Adidas Yeezy 950 Turtle Dove,NaN,-,NaN,2,https://kream.co.kr/products/5829
20182,Yeezy,Adidas,Adidas Yeezy 950 Peyote,NaN,-,NaN,8,https://kream.co.kr/products/5828
20183,Yeezy,Nike,Nike Air Yeezy 2 Solar Red,NaN,-,NaN,139,https://kream.co.kr/products/2745


In [ ]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20184 entries, 0 to 20183
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   검색어     20184 non-null  object
 1   브랜드     20184 non-null  object
 2   제품명     20184 non-null  object
 3   제품한글명   2 non-null      object
 4   가격      20184 non-null  object
 5   거래량     11694 non-null  object
 6   관심수     20184 non-null  object
 7   url     20184 non-null  object
dtypes: object(8)
memory usage: 1.9+ MB


In [ ]:
product_df.to_csv('Data/크림_브랜드별_신발_제품.csv',index=False,encoding='utf-8-sig')

In [ ]:
len(product_df)

20184